# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import pandas as pd
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split
import logging
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core import Workspace, Experiment, Dataset, Model
import numpy as np
import joblib, pickle

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
from azureml.core import Workspace

subscription_id = '6971f5ac-8af1-446e-8034-05acea24681f'
resource_group = 'aml-quickstarts-138127'
workspace_name = 'quick-starts-ws-138127'

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    ws.write_config()
    print('Library configuration succeeded')
except:
    print('Workspace not found')



Library configuration succeeded


In [6]:
from azureml.core import Workspace, Dataset

subscription_id = '6971f5ac-8af1-446e-8034-05acea24681f'
resource_group = 'aml-quickstarts-138127'
workspace_name = 'quick-starts-ws-138127'

exp = Experiment(workspace=ws, name="capstone")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-138127
Azure region: southcentralus
Subscription id: 6971f5ac-8af1-446e-8034-05acea24681f
Resource group: aml-quickstarts-138127


In [8]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '6971f5ac-8af1-446e-8034-05acea24681f'
resource_group = 'aml-quickstarts-138127'
workspace_name = 'quick-starts-ws-138127'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='train-data')

data=dataset.to_pandas_dataframe()

In [9]:
data.head(2)

,EmployeeNo,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
0,YAK/S/00001,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,2,1986,12.5,2011,1,0,41,ANAMBRA,False,Married,False,False,0,0
1,YAK/S/00002,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,ANAMBRA,True,Married,False,False,0,0


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute


from azureml.core.compute_target import ComputeTargetException

cluster_name = "capstone-ml"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('I found the existing cluster, So I am using it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           vm_priority = 'lowpriority',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

I found the existing cluster, So I am using it.


Checking for Missing information to properly clean the data

In [10]:
data.isnull().sum()

EmployeeNo                                0
Division                                  0
Qualification                          1679
Gender                                    0
Channel_of_Recruitment                    0
Trainings_Attended                        0
Year_of_birth                             0
Last_performance_score                    0
Year_of_recruitment                       0
Targets_met                               0
Previous_Award                            0
Training_score_average                    0
State_Of_Origin                           0
Foreign_schooled                          0
Marital_Status                            0
Past_Disciplinary_Action                  0
Previous_IntraDepartmental_Movement       0
No_of_previous_employers                  0
Promoted_or_Not                           0
dtype: int64

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38312 entries, 0 to 38311
Data columns (total 19 columns):
EmployeeNo                             38312 non-null object
Division                               38312 non-null object
Qualification                          36633 non-null object
Gender                                 38312 non-null object
Channel_of_Recruitment                 38312 non-null object
Trainings_Attended                     38312 non-null int64
Year_of_birth                          38312 non-null int64
Last_performance_score                 38312 non-null float64
Year_of_recruitment                    38312 non-null int64
Targets_met                            38312 non-null int64
Previous_Award                         38312 non-null int64
Training_score_average                 38312 non-null int64
State_Of_Origin                        38312 non-null object
Foreign_schooled                       38312 non-null bool
Marital_Status                         38312

In [12]:
data.columns

Index(['EmployeeNo', 'Division', 'Qualification', 'Gender',
       'Channel_of_Recruitment', 'Trainings_Attended', 'Year_of_birth',
       'Last_performance_score', 'Year_of_recruitment', 'Targets_met',
       'Previous_Award', 'Training_score_average', 'State_Of_Origin',
       'Foreign_schooled', 'Marital_Status', 'Past_Disciplinary_Action',
       'Previous_IntraDepartmental_Movement', 'No_of_previous_employers',
       'Promoted_or_Not'],
      dtype='object')

In [13]:
def year_count(variable,df):
    df[variable] = df[variable].astype("int")
    df[variable] = 2019.0 - df[variable]
    return df[variable]

def geo_zones(state):
    North_Central = ['KOGI','BENUE','KWARA','NASSARAWA','NIGER','PLATEAU','FCT']
    North_East = ['ADAMAWA','BAUCHI','BORNO','GOMBE','TARABA','YOBE']
    North_West = ['JIGAWA','KADUNA','KANO','KATSINA','KEBBI','SOKOTO','ZAMFARA']
    South_East = ['ABIA','ANAMBRA','EBONYI','ENUGU','IMO']
    South_South = ['AKWA IBOM','BAYELSA','CROSS RIVER','RIVERS','DELTA','EDO']
    South_West = ['EKITI','LAGOS','OGUN','ONDO','OSUN','OYO']
    
    if state in North_Central:
        return "North_Central"
    elif state in North_East :
        return "North_East"
    elif state in North_West:
        return "North_West"
    elif state in South_East:
        return "South_East"
    elif state in South_South:
        return "South_South"
    else:
        return "South_West"

def clean_data(data):
    
    data['Qualification'].fillna(data['Qualification'].value_counts().index[0],inplace=True)
    degree_holder ={"MSc, MBA and PhD":"Yes","Non-University Education":"No","First Degree or HND":"Yes"}
    data['Qualification'] = data['Qualification'].map(degree_holder)
    data['age_recruitment'] = year_count('Year_of_recruitment',data)
    data.drop(labels = ['Year_of_recruitment','Year_of_birth'], axis = 1, inplace = True)
    data['State_Of_Origin'] = data['State_Of_Origin'].apply(lambda x: geo_zones(x))
    data.drop(['EmployeeNo'],axis=1,inplace=True)
    column = ['Gender','Division','State_Of_Origin','Qualification','Channel_of_Recruitment','Last_performance_score','Foreign_schooled','Marital_Status','Past_Disciplinary_Action','Previous_IntraDepartmental_Movement','No_of_previous_employers']
    data=pd.get_dummies(data, columns=column,drop_first=True)

    return data
   

data = clean_data(data)

In [14]:
data.head()

,Trainings_Attended,Targets_met,Previous_Award,Training_score_average,Promoted_or_Not,age_recruitment,Gender_Male,Division_Commercial Sales and Marketing,Division_Customer Support and Field Operations,Division_Information Technology and Solution Support,...,Marital_Status_Not_Sure,Marital_Status_Single,Past_Disciplinary_Action_True,Previous_IntraDepartmental_Movement_True,No_of_previous_employers_1,No_of_previous_employers_2,No_of_previous_employers_3,No_of_previous_employers_4,No_of_previous_employers_5,No_of_previous_employers_More than 5
0,2,1,0,41,0,8.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,52,0,4.0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,42,0,7.0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,42,0,10.0,1,1,0,0,...,0,1,0,0,1,0,0,0,0,0
4,3,0,0,77,0,7.0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [15]:
x = data.drop(['Promoted_or_Not'],axis=1)
y = data['Promoted_or_Not']
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [16]:
df_train = pd.concat([x_train, y_train], axis=1)

In [17]:

df_train.head()

,Trainings_Attended,Targets_met,Previous_Award,Training_score_average,age_recruitment,Gender_Male,Division_Commercial Sales and Marketing,Division_Customer Support and Field Operations,Division_Information Technology and Solution Support,Division_Information and Strategy,...,Marital_Status_Single,Past_Disciplinary_Action_True,Previous_IntraDepartmental_Movement_True,No_of_previous_employers_1,No_of_previous_employers_2,No_of_previous_employers_3,No_of_previous_employers_4,No_of_previous_employers_5,No_of_previous_employers_More than 5,Promoted_or_Not
15268,2,0,0,49,7.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11115,2,0,0,57,9.0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
4826,3,0,0,70,6.0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
37588,3,1,0,49,7.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
37186,2,0,0,48,5.0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0


In [18]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: Promoted_or_Not, dtype: int64

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [19]:
# TODO: Put your automl settings here
automl_settings = {
    "featurization": "auto",
    "n_cross_validations": 4,
    "experiment_timeout_minutes": 30,
    "enable_early_stopping": True,
    "verbosity": logging.INFO,
}

In [22]:
# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=df_train,
    label_column_name="Promoted_or_Not",
    n_cross_validations=5)


In [23]:
# TODO: Submit your experiment
remote_run = exp.submit(automl_config,show_output=True)

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_91b7d0bd-1b7d-457b-8320-bf768ab34270

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/Auto

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [24]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [25]:
best_automl_run, fitted_automl_model = remote_run.get_output()

In [26]:
print(fitted_automl_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               tree_method='hist',
                                                                                               verbose=-10,
                                                                                               verbosity=0))],
                                                          

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [27]:
#TODO: Save the best model
best_model,fitted = remote_run.get_output()
print(best_model)


Run(Experiment: capstone,
Id: AutoML_91b7d0bd-1b7d-457b-8320-bf768ab34270_41,
Type: None,
Status: Completed)


In [28]:
print(fitted)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               tree_method='hist',
                                                                                               verbose=-10,
                                                                                               verbosity=0))],
                                                          

In [29]:
filename = 'best_automl_model.joblib'
# pickle.dump(fitted, open(filename, 'wb'))

joblib.dump(fitted,filename)

['best_automl_model.joblib']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [30]:
model = Model.register(
    workspace=ws,
    model_path="./best_automl_model.joblib",
    model_name="BestModel",
    description="Employee Promotion status prediction Model"
)

Registering model BestModel


TODO: In the cell below, send a request to the web service you deployed to test it.

In [31]:
#Create inference config
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment.get(workspace=ws,name='AzureML-AutoML')

inferencefinal = InferenceConfig(entry_script='score.py',environment=env) 

#Deploy model and check service state
from azureml.core.webservice import LocalWebservice, AciWebservice


deployfinal = AciWebservice.deploy_configuration()


TODO: In the cell below, print the logs of the web service and delete the service

In [32]:
svcs = Model.deploy(workspace=ws,name='deploy',models=[model],inference_config=inferencefinal,deployment_config=deployfinal)
svcs.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [33]:
print(svcs.state)


Healthy


In [34]:

print(svcs.scoring_uri)

http://b26582e9-cb54-436d-b5a0-ed7cde6f5c28.southcentralus.azurecontainer.io/score


Extracting a sample data from the train data to test the deployed model

In [35]:
x_test.iloc[0].to_dict()

{'Trainings_Attended': 2.0,
 'Targets_met': 0.0,
 'Previous_Award': 0.0,
 'Training_score_average': 53.0,
 'age_recruitment': 4.0,
 'Gender_Male': 0.0,
 'Division_Commercial Sales and Marketing': 0.0,
 'Division_Customer Support and Field Operations': 1.0,
 'Division_Information Technology and Solution Support': 0.0,
 'Division_Information and Strategy': 0.0,
 'Division_People/HR Management': 0.0,
 'Division_Regulatory and Legal services': 0.0,
 'Division_Research and Innovation': 0.0,
 'Division_Sourcing and Purchasing': 0.0,
 'State_Of_Origin_North_East': 0.0,
 'State_Of_Origin_North_West': 0.0,
 'State_Of_Origin_South_East': 0.0,
 'State_Of_Origin_South_South': 0.0,
 'State_Of_Origin_South_West': 1.0,
 'Qualification_Yes': 1.0,
 'Channel_of_Recruitment_Direct Internal process': 1.0,
 'Channel_of_Recruitment_Referral and Special candidates': 0.0,
 'Last_performance_score_2.5': 0.0,
 'Last_performance_score_5.0': 0.0,
 'Last_performance_score_7.5': 1.0,
 'Last_performance_score_10.0':

In [45]:
import requests
import json

# URL for the web service
scoring_uri = 'http://b26582e9-cb54-436d-b5a0-ed7cde6f5c28.southcentralus.azurecontainer.io/score'


data = {'data':[ {'Trainings_Attended': 2.0,
 'Targets_met': 0.0,
 'Previous_Award': 0.0,
 'Training_score_average': 53.0,
 'age_recruitment': 4.0,
 'Gender_Male': 0.0,
 'Division_Commercial Sales and Marketing': 0.0,
 'Division_Customer Support and Field Operations': 1.0,
 'Division_Information Technology and Solution Support': 0.0,
 'Division_Information and Strategy': 0.0,
 'Division_People/HR Management': 0.0,
 'Division_Regulatory and Legal services': 0.0,
 'Division_Research and Innovation': 0.0,
 'Division_Sourcing and Purchasing': 0.0,
 'State_Of_Origin_North_East': 0.0,
 'State_Of_Origin_North_West': 0.0,
 'State_Of_Origin_South_East': 0.0,
 'State_Of_Origin_South_South': 0.0,
 'State_Of_Origin_South_West': 1.0,
 'Qualification_Yes': 1.0,
 'Channel_of_Recruitment_Direct Internal process': 1.0,
 'Channel_of_Recruitment_Referral and Special candidates': 0.0,
 'Last_performance_score_2.5': 0.0,
 'Last_performance_score_5.0': 0.0,
 'Last_performance_score_7.5': 1.0,
 'Last_performance_score_10.0': 0.0,
 'Last_performance_score_12.5': 0.0,
 'Foreign_schooled_True': 1.0,
 'Marital_Status_Not_Sure': 0.0,
 'Marital_Status_Single': 1.0,
 'Past_Disciplinary_Action_True': 0.0,
 'Previous_IntraDepartmental_Movement_True': 0.0,
 'No_of_previous_employers_1': 0.0,
 'No_of_previous_employers_2': 0.0,
 'No_of_previous_employers_3': 0.0,
 'No_of_previous_employers_4': 0.0,
 'No_of_previous_employers_5': 0.0,
 'No_of_previous_employers_More than 5': 0.0}
]
        }

input_data = json.dumps(data)

headers = {'Content-Type': 'application/json'}
# # Make the request and display the response
result = requests.post(scoring_uri,input_data,headers=headers)
print("prediction:",result.text)



prediction: [1]


In [41]:
env.save_to_directory('./training2',overwrite=False)

In [40]:
print(svcs.get_logs())

2021-02-08T10:26:43,787448575+00:00 - gunicorn/run 
2021-02-08T10:26:43,884206255+00:00 - rsyslog/run 
2021-02-08T10:26:44,083411255+00:00 - iot-server/run 
rsyslogd: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libuuid.so.1: no version information available (required by rsyslogd)
2021-02-08T10:26:44,184680967+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml

In [62]:
svcs.delete()